# North Carolina Scorecard

This notebook generates the county, school district, and statewide "future voter scorecards" for NC. It is generalize to be updated every month, with minimal changes.

Scorecard outputs (tables) are written back to BigQuery, where they are then read into Google Sheets for formatting

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq

## Inputs
Update the fields below each month

In [2]:
# Inputs
as_of_data_date = pd.Timestamp("2025-09-20")
acs_year = '2022' # 2022 for 2024 scorecards. ACS vintages trail by 2 years


## Outputs
Run the cells below, without edits each month

In [3]:
# Derived variables
latest_18_yob = as_of_data_date.year - 18 # 2006 for 2024 scorecards
earliest_18_yob = latest_18_yob - 1 # 2005 for 2024 scorecards

latest_45_yob = as_of_data_date.year - 45 # 1979 for 2024 scorecards
earliest_45_yob = latest_45_yob - 1 # 1978 for 2024 scorecards

data_date_suffix = str(as_of_data_date.year) + str(as_of_data_date.month).rjust(2, "0") + str(as_of_data_date.day).rjust(2, "0")  # this is the rolling "as of" date, where we snap the line for 18yos

# Define columns

    # Voter file
REG_YOB_LATE_YEAR = 'REG_YOB_' + str(latest_18_yob) # col name for number of registrants with given birth year (2006 for 2024 scorecards)
REG_YOB_EARLY_YEAR = 'REG_YOB_' + str(earliest_18_yob) # col name for number of registrants with given birth year (2005 for 2024 scorecards)

REG_45_PLUS_YOB_LATE_YEAR = 'REG_45_PLUS_YOB_' + str(latest_45_yob) # col name for number of registrants with given birth year (1979 for 2024 scorecards)
REG_45_PLUS_YOB_EARLY_YEAR = 'REG_45_PLUS_YOB_' + str(earliest_45_yob) # col name for number of registrants with given birth year (1978 for 2024 scorecards)

    # ACS
EST_18_YO_THIS_YEAR = 'EST_18_YO_' + str(as_of_data_date.year) # col name for number of 18 yos this "as_of" year estimated from ACS
EST_18_AND_19_YO_THIS_YEAR = 'EST_18_AND_19_YO_' + str(as_of_data_date.year) # col name for number of 18 and 19 yos this "as_of" year estimated from ACS

EST_45_PLUS_YO_THIS_YEAR = 'EST_45_PLUS_YO_' + str(as_of_data_date.year) # col name for number of 45+ yos in the "as of" year estimated from ACS


### County Scorecard

In [4]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
voter_source_table = data_date_suffix + "_nc_voter_registration_and_history"

acs_S0101_table = "S0101_us_counties_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file by county, counting the number of registrants in a given birth year.
* Then, left joins the county estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos are derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [5]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH addresses AS(
    SELECT
    ncid as VOTER_ID,
    CASE 
        -- WATAUGA 
        WHEN county_id = '95' AND (mail_addr1 LIKE 'ASU %') THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
        -- ORANGE 
        WHEN county_id = '68' AND res_city_desc = 'CHAPEL HILL' THEN FARM_FINGERPRINT(CONCAT(COALESCE(REGEXP_REPLACE(res_street_address, '( #.*)$', '') ,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
   
        ELSE FARM_FINGERPRINT(CONCAT(COALESCE(res_street_address,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
    END as ADDRESS_ID
    FROM `tcc-research.nc_sources.""" + voter_source_table + """`


), young_voters AS(
    
    SELECT  
    a.*,
    ADDRESS_ID 
    FROM `tcc-research.nc_production.""" + voter_file_table + """` a
    LEFT JOIN addresses on addresses.VOTER_ID = a.VOTER_ID
    WHERE VOTER_STATUS IN ('ACTIVE')

),address_count_18 AS(    
    SELECT  
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
    FROM young_voters a
    WHERE YEAR_OF_BIRTH IN (""" + str(latest_18_yob) + ", " + str(earliest_18_yob) + """)
    GROUP BY ADDRESS_ID

),voter_file_county AS(
    SELECT
    COUNTY_FIPS,
    COUNTY_NAME,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = """ + str(latest_18_yob) + ") AS " + REG_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH = """ + str(earliest_18_yob) + ") AS " + REG_YOB_EARLY_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(latest_45_yob) + ") AS " + REG_45_PLUS_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(earliest_45_yob) + ") AS " + REG_45_PLUS_YOB_EARLY_YEAR + """,
    FROM young_voters
    LEFT JOIN address_count_18 on young_voters.ADDRESS_ID = address_count_18.ADDRESS_ID
    WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0)<4
    GROUP BY COUNTY_FIPS, COUNTY_NAME

), acs_county AS(
    SELECT
    COUNTY_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS """ + EST_18_YO_THIS_YEAR + """,
    EST_15_TO_17_YO * 2 / 3 AS """ + EST_18_AND_19_YO_THIS_YEAR + """,
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
    FROM `tcc-research.acs_sources.""" + acs_S0101_table + """`
    WHERE STATE_FIPS = "37"

)

SELECT
voter_file_county.*,
acs_county.EST_15_TO_17_YO,
acs_county.MOE_15_TO_17_YO,
acs_county.""" + EST_18_YO_THIS_YEAR + """,
acs_county.""" + EST_18_AND_19_YO_THIS_YEAR + """,
acs_county.""" + EST_45_PLUS_YO_THIS_YEAR + """

FROM voter_file_county LEFT JOIN acs_county ON voter_file_county.COUNTY_FIPS = acs_county.COUNTY_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [6]:
# Preview
df.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,089,HENDERSON,78424,613,1002,54429,53485,3991,56,1330.333333,2660.666667,70710
1,119,MECKLENBURG,683266,7069,10706,347112,335955,42911,18,14303.666667,28607.333333,465789
2,003,ALEXANDER,23044,277,341,14995,14702,1341,121,447.000000,894.000000,20246
3,065,EDGECOMBE,29375,242,447,18826,18419,2015,25,671.666667,1343.333333,26060
4,083,HALIFAX,27682,242,437,18576,18173,1882,88,627.333333,1254.666667,27169


In [7]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [8]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.2774725274725275


In [9]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [10]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250920,EST_REG_RATE_18_YO_AS_OF_20250920
44,183,WAKE,740355,8689,12815,400977,388094,47558,30,15852.666667,31705.333333,499266,12244.81044,0.772413
1,119,MECKLENBURG,683266,7069,10706,347112,335955,42911,18,14303.666667,28607.333333,465789,10039.620879,0.701891
13,081,GUILFORD,325531,3199,5348,190284,185406,21093,*****,7031.000000,14062.000000,253633,4682.923077,0.666039
38,067,FORSYTH,228636,2384,3665,135769,132531,15559,27,5186.333333,10372.666667,186737,3400.936813,0.65575
39,063,DURHAM,197348,1758,2582,100507,97538,10224,39,3408.000000,6816.000000,139455,2474.434066,0.726066


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [11]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.7205479452054795


In [12]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [13]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head(20)

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250920,EST_REG_RATE_18_YO_AS_OF_20250920,EST_REG_45_PLUS_YO_AS_OF_20250920,EST_REG_RATE_45_PLUS_YO_AS_OF_20250920
44,183,WAKE,740355,8689,12815,400977,388094,47558,30,15852.666667,31705.333333,499266,12244.81044,0.772413,397376.819178,0.795922
1,119,MECKLENBURG,683266,7069,10706,347112,335955,42911,18,14303.666667,28607.333333,465789,10039.620879,0.701891,343994.153425,0.738519
13,081,GUILFORD,325531,3199,5348,190284,185406,21093,*****,7031.000000,14062.000000,253633,4682.923077,0.666039,188920.832877,0.744859
38,067,FORSYTH,228636,2384,3665,135769,132531,15559,27,5186.333333,10372.666667,186737,3400.936813,0.65575,134864.134247,0.722214
39,063,DURHAM,197348,1758,2582,100507,97538,10224,39,3408.000000,6816.000000,139455,2474.434066,0.726066,99677.306849,0.714763
7,021,BUNCOMBE,179475,1504,2118,109997,107088,8663,38,2887.666667,5775.333333,144295,2091.686813,0.724352,109184.073973,0.756673
58,051,CUMBERLAND,172924,2026,3047,97078,94405,12860,36,4286.666667,8573.333333,129853,2871.458791,0.669858,96331.024658,0.741847
76,179,UNION,157716,2505,3422,94373,91458,12970,5,4323.333333,8646.666667,115787,3454.510989,0.799039,93558.39726,0.808022
31,129,NEW HANOVER,156410,1305,2155,93613,91380,7279,81,2426.333333,4852.666667,115498,1902.953297,0.784292,92988.983562,0.805113
59,101,JOHNSTON,144065,1728,2672,82781,80316,9984,98,3328.000000,6656.000000,103011,2469.406593,0.742009,82092.150685,0.796926


In [15]:
df_reg_est = df_reg_est.sort_values('EST_REG_RATE_18_YO_AS_OF_20250920', ascending=False)
df_reg_est

,COUNTY_FIPS,COUNTY_NAME,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250920,EST_REG_RATE_18_YO_AS_OF_20250920,EST_REG_45_PLUS_YO_AS_OF_20250920,EST_REG_RATE_45_PLUS_YO_AS_OF_20250920
42,189,WATAUGA,34300,289,467,17708,17279,1202,29,400.666667,801.333333,23535,418.57967,1.044708,17588.115068,0.747317
20,177,TYRRELL,1866,6,21,1345,1316,36,39,12.000000,24.000000,1839,11.826923,0.985577,1336.89589,0.726969
95,141,PENDER,44946,553,723,28316,27532,2379,130,793.000000,1586.000000,32899,753.612637,0.950331,28096.909589,0.854035
73,019,BRUNSWICK,125097,787,1125,95394,94211,3532,58,1177.333333,2354.666667,97646,1099.156593,0.933598,95063.408219,0.973551
36,031,CARTERET,50896,495,677,36190,35557,2231,114,743.666667,1487.333333,42921,682.848901,0.918219,36013.106849,0.839056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,145,PERSON,24602,146,347,16301,15958,1476,87,492.000000,984.000000,21962,242.282967,0.492445,16205.147945,0.737872
54,173,SWAIN,8255,59,111,5469,5332,584,98,194.666667,389.333333,7158,89.799451,0.461299,5430.715068,0.758692
41,033,CASWELL,13471,72,159,9218,9056,783,46,261.000000,522.000000,13432,116.118132,0.444897,9172.728767,0.682901
74,103,JONES,6111,32,78,4169,4069,392,71,130.666667,261.333333,5813,53.642857,0.410532,4141.054795,0.712378


#### Output
Write back to BQ

##### Wide

In [16]:
# Flag largest counties
df_reg_est['is_in_10_largest'] = np.where(df_reg_est.COUNTY_NAME.isin(df_reg_est.nlargest(10, columns='EST_18_YO_2025').COUNTY_NAME),1,0)

In [17]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_county_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]


### Statewide Scorecard

In [18]:
# Define table names
voter_file_table = data_date_suffix + "_scorecard_nc"
acs_S0101_table = "S0101_us_states_acs5y_" + acs_year

#### Query from BQ
This query:
* Summarizes the voter file for NC state, counting the number of registrants in a given birth year.
* Then, left joins the statewide estimates for the total number of 18 (and 19) yos from the ACS
    * The estimates for the total number of 18 (and 19) yos is derived from the raw estimates of 15-17 yos, **assuming a uniform distribution of population across 15, 16, and 17 year olds.**
    * Since the ACS trails by 2 years, the ACS estimate of 15-17yos is used as a proxy for the number of 17-19yos today. (This means we are intentionally *not* trying to count the college student or "group quarters" population in our denominator)

In [19]:
# Define GCP project
project_id = "tcc-research"

# Define query, including variables and column names that adjust with time
sql = """
WITH addresses AS(
    SELECT
    ncid as VOTER_ID,
    CASE 
        -- WATAUGA 
        WHEN county_id = '95' AND (mail_addr1 LIKE 'ASU %') THEN FARM_FINGERPRINT('APPALACHIAN STATE UNIVERSITY DORM')
        -- ORANGE 
        WHEN county_id = '68' AND res_city_desc = 'CHAPEL HILL' THEN FARM_FINGERPRINT(CONCAT(COALESCE(REGEXP_REPLACE(res_street_address, '( #.*)$', '') ,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
   
        ELSE FARM_FINGERPRINT(CONCAT(COALESCE(res_street_address,''), COALESCE(res_city_desc,''), COALESCE(state_cd,''), COALESCE(zip_code,'')))
    END as ADDRESS_ID
    FROM `tcc-research.nc_sources.""" + voter_source_table + """`


), young_voters AS(
    
    SELECT  
    a.*,
    ADDRESS_ID 
    FROM `tcc-research.nc_production.""" + voter_file_table + """` a
    LEFT JOIN addresses on addresses.VOTER_ID = a.VOTER_ID
    WHERE VOTER_STATUS IN ('ACTIVE')

),address_count_18 AS(    
    SELECT  
    ADDRESS_ID,
    COUNT(VOTER_ID) AS N_18_VOTERS_AT_ADDRESS
    FROM young_voters a
    WHERE YEAR_OF_BIRTH IN (""" + str(latest_18_yob) + ", " + str(earliest_18_yob) + """)
    GROUP BY ADDRESS_ID

),voter_file_nc AS(
    SELECT
    STATE_FIPS,
    COUNT(VOTER_ID) AS N_VOTERS,
    COUNTIF(YEAR_OF_BIRTH = """ + str(latest_18_yob) + ") AS " + REG_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH = """ + str(earliest_18_yob) + ") AS " + REG_YOB_EARLY_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(latest_45_yob) + ") AS " + REG_45_PLUS_YOB_LATE_YEAR + """,
    COUNTIF(YEAR_OF_BIRTH <= """ + str(earliest_45_yob) + ") AS " + REG_45_PLUS_YOB_EARLY_YEAR + """,
    FROM young_voters
    LEFT JOIN address_count_18 on young_voters.ADDRESS_ID = address_count_18.ADDRESS_ID
    WHERE COALESCE(N_18_VOTERS_AT_ADDRESS, 0)<4
    GROUP BY STATE_FIPS

), acs_nc AS(
    SELECT
    STATE_FIPS,
    EST_15_TO_17_YO,
    MOE_15_TO_17_YO,
    EST_15_TO_17_YO / 3 AS """ + EST_18_YO_THIS_YEAR + """,
    EST_15_TO_17_YO * 2 / 3 AS """ + EST_18_AND_19_YO_THIS_YEAR + """,
    EST_45_TO_49_YO + EST_50_TO_54_YO + EST_55_TO_59_YO + EST_55_TO_59_YO + EST_60_AND_OVER AS """ + EST_45_PLUS_YO_THIS_YEAR + """
    FROM `tcc-research.acs_sources.""" + acs_S0101_table + """`
    WHERE STATE_FIPS = "37"

)

SELECT
voter_file_nc.*,
acs_nc.EST_15_TO_17_YO,
acs_nc.MOE_15_TO_17_YO,
acs_nc.""" + EST_18_YO_THIS_YEAR + """,
acs_nc.""" + EST_18_AND_19_YO_THIS_YEAR + """,
acs_nc.""" + EST_45_PLUS_YO_THIS_YEAR + """

FROM voter_file_nc LEFT JOIN acs_nc ON voter_file_nc.STATE_FIPS = acs_nc.STATE_FIPS
"""
# Query
df = pandas_gbq.read_gbq(sql, project_id=project_id)

Downloading: 100%|██████████|


In [20]:
# Preview
df

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025
0,37,6521238,67322,104013,3900056,3803275,404849,794,134949.666667,269899.333333,5139416


In [21]:
df_reg_est = df.copy()

#### Metric 1: Estimated registration rate of 18 year olds as of a rolling date (i.e. latest month)
Ex: In March 2024, we consider the registration rate among those born between March 2nd 2005 and March 1st 2006


Notes:
- The MI voter file does *not* include full birth dates for registrants – only year of birth is included
- 18 yos as of a given date in the middle of the calendar year can have 2 potential years of birth. We refer to these as the "later 18 yo year of birth" (2006 for 2024 scorecards) and the "earlier 18 yo year of birth" (2005 for 2024 scorecards)
- MI voter file includes 18yo registrants only: Those who are 18 years old prior to March 1st 2024. This was confirmed by the SOS. This means we can assume everyone born in the "later 18 year old year" is 18 (and there are no 17 year old).
- We still need to discount those born in the "earlier 18-year old year", because some of those born in that year are already 19
    - Ex: in March 2024, those born in Jan (31 days) and Feb (28 days) 2005 are already 19, so only those born March through December 2005 are 18

Estimation:

To estimate the number of 18 yos as of a rolling date, we "pro-rate" the number of registrants born in a given year based on the share of days in the year that could be 18yo birthdays. There are two steps:
- For the later 18 yo year of birth: Assume all are 18
- For the earlier 18 yo year of birth: Estimate the number of days that could be 18yo birthdays. Calculate the number of total potential birthdays included in the voter file (just ~365). Calculate the ratio of these numbers.


Assumptions:
- Voter file is as of 1st of the month (confirmed by SOS)
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 18 yos, and younger 19yos

In [24]:
# Define column names
EST_REG_18_YO_AS_OF_ROLLING = 'EST_REG_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date

# Birthday splits 18 yo vs. 19 yo in voter file (hypothetical)
earliest_bday_18 = as_of_data_date - pd.tseries.offsets.DateOffset(years=19) + pd.tseries.offsets.DateOffset(days=1) # earliest possible bday for 18yo

n_bdays_of_18_early = pd.Timestamp(str(earliest_18_yob) +"-12-31") - earliest_bday_18 # number of possible bdays of 18 yos in earlier 18 yo year of birth
n_total_days_of_early_year = pd.Timestamp(str(earliest_18_yob) +"-12-31") - pd.Timestamp(str(earliest_18_yob) +"-01-01") # number of total birthdays in earlier 18 yo year of birth (should be 365)

# Discounts
    # Share of 18yo in late year
share_18_late_year = 1

    # Share of 18yo in early year
share_18_early_year = n_bdays_of_18_early / n_total_days_of_early_year

    # CHECKS
print("share of 18 yo in late year: {}".format(share_18_late_year))
print("share of 18 yo in early year: {}".format(share_18_early_year))

share of 18 yo in late year: 1
share of 18 yo in early year: 0.2774725274725275


In [25]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] = df_reg_est[REG_YOB_LATE_YEAR] * share_18_late_year  + df_reg_est[REG_YOB_EARLY_YEAR] * share_18_early_year

# Calculate estimated registration rate
EST_REG_RATE_18_YO_AS_OF_ROLLING = 'EST_REG_RATE_18_YO_AS_OF_' + data_date_suffix # col name for estimated 18yo as of rolling date
df_reg_est[EST_REG_RATE_18_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_18_YO_AS_OF_ROLLING] / df_reg_est[EST_18_YO_THIS_YEAR] # estimated registered 18yo over ACS 18yo population estimate

In [26]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250920,EST_REG_RATE_18_YO_AS_OF_20250920
0,37,6521238,67322,104013,3900056,3803275,404849,794,134949.666667,269899.333333,5139416,96182.75,0.712731


#### Metric 2: Estimated registration rate of 45 year olds as of a rolling date (i.e. latest month)
To count the 45+ yo as of a rolling date, we need to discount some folks born in the latest year of 45 year olds, because they are still 44

Assumptions:
- Even distribution of birthdays across all days of year
- Uniform registration rates among older 44 yos, and younger 45yos

In [27]:
# Define column names
EST_REG_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date

# Birthday splits 44 yo vs. 45 yo in voter file (hypothetical)
lastest_bday_45 = as_of_data_date - pd.tseries.offsets.DateOffset(years=45)  # latest possible bday for 45yo

n_bdays_of_45 = lastest_bday_45 - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of possible bdays of 45 yos in later 45yo year of birth
n_total_days_of_late_year = pd.Timestamp(str(latest_45_yob) +"-12-31") - pd.Timestamp(str(latest_45_yob) +"-01-01") # number of total birthdays in later 45yo year of birth


    # Share of 45yo in early year
share_45_late_year = n_bdays_of_45 / n_total_days_of_late_year

    # CHECKS
print("share of 45 yo in early year: {}".format(share_45_late_year))

share of 45 yo in early year: 0.7205479452054795


In [28]:
# Calculate numerator (registrants)
df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] = ((df_reg_est[REG_45_PLUS_YOB_LATE_YEAR] - df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]) * share_45_late_year)  + df_reg_est[REG_45_PLUS_YOB_EARLY_YEAR]

# Calculate estimated registration rate
EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING = 'EST_REG_RATE_45_PLUS_YO_AS_OF_' + data_date_suffix # col name for estimated 45yo as of rolling date
df_reg_est[EST_REG_RATE_45_PLUS_YO_AS_OF_ROLLING] = df_reg_est[EST_REG_45_PLUS_YO_AS_OF_ROLLING] / df_reg_est[EST_45_PLUS_YO_THIS_YEAR] # estimated registered 45yo over ACS 45yo population estimate

In [29]:
df_reg_est = df_reg_est.sort_values('N_VOTERS', ascending=False)
df_reg_est.head()

,STATE_FIPS,N_VOTERS,REG_YOB_2007,REG_YOB_2006,REG_45_PLUS_YOB_1980,REG_45_PLUS_YOB_1979,EST_15_TO_17_YO,MOE_15_TO_17_YO,EST_18_YO_2025,EST_18_AND_19_YO_2025,EST_45_PLUS_YO_2025,EST_REG_18_YO_AS_OF_20250920,EST_REG_RATE_18_YO_AS_OF_20250920,EST_REG_45_PLUS_YO_AS_OF_20250920,EST_REG_RATE_45_PLUS_YO_AS_OF_20250920
0,37,6521238,67322,104013,3900056,3803275,404849,794,134949.666667,269899.333333,5139416,96182.75,0.712731,3873010.350685,0.75359


#### Output
Write back to BQ

In [30]:
# write
project_id = "tcc-research"
table_id = 'nc_output.' + data_date_suffix+ '_nc_statewide_scorecard_output'

pandas_gbq.to_gbq(df_reg_est, table_id, project_id=project_id, if_exists='replace')

100%|██████████| 1/1 [00:00<?, ?it/s]
